# Tugas Prkatikum 4 DL 2021
---
M. Naufal Irham R (10818019)

## Import library dan upload data ke google colab

In [33]:
import numpy as np
import pandas as pd
np.set_printoptions(precision = 3, suppress = True, formatter = {'float':'{:6.5f}'.format})

Import manual datanya.

Data yang digunakan adalah komentar 2 produk di Tokopedia yaitu produk fashion dan produk teknologi.
Kenapa saya justru mengolah data komentar suatu barang? Karena pada komentar tersebut banyak sekali kata slang, ada juga yang pakai kata baku, dan banyak juga kata-kata yang typo. Jadi tipe kalimat yang diolah disini lebih beragam

Pemilihan komentar dari 2 produk berbeda pun dilakukan sengaja agar algoritmanya tidak hanya terbatas untuk satu topik saja

sumber data dari:
- https://www.tokopedia.com/copperindonesia/iphone-7-plus-8-plus-copper-tempered-glass-full-glue-premium-glossy-putih?whid=0
- https://www.tokopedia.com/kaospolos47/kaos-polos-cotton-combed-30s-premium-pria-wanita-harga-grosir-s-m-l-xl-putih-s

In [34]:
from google.colab import files
uploaded = files.upload()

## Siapkan datanya

In [35]:
f = open("Komentar.txt", "r")
data = f.read().lower().replace(",",'').replace(".",'').replace("?",'') #Disini data masih mentah dan tipenya string
#Data diatas udah dihapus tanda bancanya

kumpulan_kalimat = data.split('\n') #Dari data mentah, kita mengambil masing2 kalimat kedalam list. kebetulan antar komentar dipisahkan oleh \n
while (kumpulan_kalimat.count("")): #Kebetulan, kumpulan kalimat yang didapat, terdapat element list yang isinya cuma "", jadi dihapus dulu.
    kumpulan_kalimat.remove("") 

Untuk masuk proses selanjutnya kita butuh kumpulan kata apa saja yang ada di dalam komentar tersebut

In [36]:
kumpulan_kata = list(set(data.replace("\n",' ').split()))

Kini kita punya kumpulan kalimat dan juga kumpulan kata. Sebenarnya dari kumpulan kata tersbut, mungkin kita bisa lakukan lematisasi, stemming, dan mengubah kata typo menjadi data yang sebenarnya. Namun disini saya tidak lakukan hal itu karena beberapa hal, yaitu:
- Sulit untuk dilakukan tanpa adanya bantuan libary pengolah kata
- Kalaupun saya bisa membuat algoritmanya sendiri, sepertinya kurang worth it untuk dilakukan karena mungkin hasil akhirnya pun tidak akan jauh berbeda 

## Pembobotan dilakukan dengan menggunakan TF-IDF

Disini, kita akan melakukan pembobotan pada semua kata yang ada menggunakan TF-IDF agar dapat mencari mana kata yang paling signifikan dari tiap komentar

In [37]:
#Menghitung term frequency tiap kalimat pada data

TF = []

check = kumpulan_kata
for kalimat in kumpulan_kalimat:
  ans = []
  for j in range(len(check)):
    ans.append(kalimat.count(check[j])/len(kalimat.split()))
  TF.append(ans)

#Menghitung IDF pada data
IDF = []
for j in range(len(check)):
  freq = 0
  for kalimat in kumpulan_kalimat:
    if kalimat.count(check[j]) > 0:
      freq += 1
  IDF.append(np.log(len(kumpulan_kalimat)/freq))

bobot = np.sum(np.array(TF)*IDF, axis = 0)


Setelah dilakukan pembobotan, maka selanjutnya bobot dipasangkan dengan kata yang bersangkutan

In [38]:
#Tiap kata dipasangkan dengan bobotnya masing2
kata_bobot = pd.DataFrame([kumpulan_kata, bobot]).T
kata_bobot.sort_values(1, axis=0, ascending=False)[0:60] #Kita ingin coba lihat 60 kata apa yang paling signifikan pada data tersebut

,0,1
129,barang,1.26241
335,gan,1.11481
290,sesuai,1.06122
46,ga,0.992369
45,uk,0.984441
221,it,0.96141
130,di,0.922608
153,baik,0.918809
302,ka,0.913404
62,dengan,0.904622


Setelah dilihat dari bobot diatas, kita cari mana diantara kata-kata tersebut yang dapat kita labeli sebagai kata kunci negatif dan positif

## Hasil Akhir

Setelah kita tahu apa saja kata positif dan negatif pada data, kita akan mengevaluasi komentar-komentar yang ada dengan cara mencari apakah dalam komentar tersebut terdapat kata-kata yang sudah dilabeli positif dan negatif. 
Untuk kata negatif, kita ubah bobotnya menjadi negatif, sedangkan kata positif kita ubah bobotnya menjadi positif

Setelah itu, kita hitung total bobot keseluruhan kata-kata yang bersangkutan dalam suatu komentar. Lalu bobot akhir dari suatu kalimat akan kita lihat, jika total bobot kalimat tersebut negatif, maka kalimat tersebut bernada negatif, sedangkan jika total bobot kalimat tersebut positif, maka kalimat tersebut bernada positif.

In [43]:
negative = ["lelet", "tapi", "bukan", "gak", "cuma"]
positive = ["bagus", "makasih", "cepat", "selamat", "cepet", "sesuai"]

hasil = []

for kalimat in kumpulan_kalimat:
  ans = 0
  for kata in negative:
    ans -= kalimat.count(kata)*(kata_bobot[kata_bobot[0] == kata][1]).item()
  for kata in positive:
    ans += kalimat.count(kata)*(kata_bobot[kata_bobot[0] == kata][1]).item()
  hasil.append(ans)


print("kalimat bernada negatif")
for i in range(len(hasil)):
  if hasil[i] < 0:
    print("-", kumpulan_kalimat[i])

print("\nkalimat bernada positif")
for i in range(len(hasil)):
  if hasil[i] > 0:
    print("-", kumpulan_kalimat[i])

kalimat bernada negatif
- size l bukan 75cm panjangnya tapi 67cm yah label tidak di tengah jahita kurang rapih
- waktu 1 kl beli kain bgs & uknya sesuai unt prmbelian yg 2 sebnyk 3 pc kainnya ke3 nya tidak sm uk xl sm kayak uk l sellernya gak bisa jaga kualitas kapok belanja di sini sudah saya chat supaya kirim warna hitam cuma 1 sisanya warna lain tapi tetap aja dikirim semua warna hitam lol tapi gapapa aku maklumi lah
- seler response bagus cuma kurirnya yg lelet
- lumayan lah dengan harga segitu gak usah expect terlalu tinggi
- barang bagus dan sesuai dengan deskripsi tapi kayaknya saya pesan beda warna tetapi dikirim warna yang sama tapi nggak masalah
- ini merk copper tapi kok dateng nya merk violet ya  dgn harga 47500 hrg violet temp glass 35000
- mereknya bukan copper tapi violet gak sesuai iklan
- tempered glass cukup tebal hanya saja masih agak garing sehingga gampang retak full cover-nya jg tidak fully cover terutama untuk bagian pinggir atas dan bawah kanan dan kiri tapi not

## Kesimpulan

- Dari hasil diatas, kita dapat melihat pembagian mana kalimat yang bernada positif, mana yang bernada negatif, walaupun hasilnya masih ada beberapa yang salah klasifikasinya

- Kata kunci yang didapat dari keseluruhan data tersebut adalah:
  - Negative = ["lelet", "tapi", "bukan", "gak", "cuma"]
  - Positive = ["bagus", "makasih", "cepat", "selamat", "cepet", "sesuai"]

- Hasil dari algoritma ini pasti masih banyak kekurangannya. Algoritma ini masih terlalu simple, apalagi algoritma ini tidak dibantu dengan library apapun kecuali numpy dan pandas. Maka dari itu, jika dapat digunakan library dan algoritma yang lebih kompleks (seperti LSTM), mungkin hasilnya akan lebih baik 